In [19]:
import numpy as np
import pandas as pd
import tkinter as tk
from tkinter import ttk
from tkinter import filedialog
from tkinter import *
from tkinter.ttk import *
import xlsxwriter
import re
import os

#for fixing window width
pd.options.display.width = 0

#for accessing data sorting data

motorName = 'HK-A40-10S'

selectAltitude = True         #global variable, False means use Density 

altitude = 0   #in feet
h = altitude/3.281 #convert to meters
Temperature = 15+273.15-6.5*h/1000 #temp in Kelvin
T = Temperature 
density = 1.225

#throttle information
Throttle = 100       #throttle value from 0 to 100

#esc information
eCurrMax = 180     #max current of esc
eRes = 0.004      #esc resistance in ohms

#battery cell information --- specified from Li-Pos
cells = 4           #number of cells in Li-Po battery
battVolt = 3.7      #Li-Po per-cell Voltage
battRes = 0.0028    #battery Resistance in ohms

#motor information
kV = 710           #RPM per volt for motor 
nLCurr = 1.56       #no load current in amps
mRes = 0.022         #motor resistance in ohms
mWatt = 1500          #max watts of motor

#not used, maybe can implement in future
nLVolt = 8.4        #no load voltage in Volts

velocityStep = 1.524

#add some parameters to motor name
motorName = motorName + '_' + str(kV) + 'kV_Batt' + str(cells)  + 'S'

#'''
n= 1
if n == 1:
    n=0 
    filename = "PER3_15x4E.dat"
    
#'''

#for filename in files: #add in feature later to read in multiple files and do analysis in one script

    #for reading in file automatically and determining prop diameter + prop pitch
    #---------------------------------------------------------------------#
    
    firstword = "PER3_"
    divider = "x"

    rx_to_last = r'^.*{}'.format(re.escape(divider))

    #remove PER3_ from filename
    removeBeginning = re.sub(r'^.*?'+firstword, '', filename)

    #remove .dat from filename
    propType = removeBeginning[:len(removeBeginning) - 4]
    #remove paranthesis from propType, messes up regular expressions
    propType = re.sub(r'\([^)]*\)', '', propType)
    
    propPitch = re.sub(r'^.*?'+divider, '', propType)
    
    propDiameter = (int)(re.sub(divider+propPitch, '', propType))
    #---------------------------------------------------------------------#

    #print(filename)   #print filename if having problems

    #set column index to help import data
    header_list =  [str(x) for x in range(12 + 1)]

    #read in file
    propdata = pd.read_csv(filename, sep='\s+', names=header_list)
    propdata = propdata.to_numpy()
    df = pd.DataFrame(propdata)
    df.to_csv(index=False)

    #find all rows with rpm data
    searchString = "RPM"
    mask = np.column_stack([df[col].str.contains(r''+searchString, na=False) for col in df])
    RPMdata = pd.DataFrame.dropna(df.loc[mask.any(axis=1)],axis=1)

    #create new table called RPM values to be able to bin RPM values into groups
    #remove characters from PROP RPM and change to numeric values to make analysis easier
    #first find column with numbers by converting all columns to numbers
    RPMs = RPMdata.apply(pd.to_numeric,errors = 'ignore')

    #then remove all non-int64 columns
    RPMs = RPMs.select_dtypes(include=['int64'])

    #get first RPM value index
    a = RPMs.columns.values           #weird name, maybe switch later, a[] is for locating which column RPM values at
    indexminRPM = RPMs[a[0]].index[0]

    #delete unnecessary top rows of RPM data using first RPM value
    RPMdf = df.iloc[indexminRPM:]

    #get max RPM and index
    maxRPM = RPMs.max()
    indexmaxRPM = RPMs[a[0]].idxmax()

    #get index of each RPM value to bin data
    RPMindex = RPMs.index

    #clear out columns with mostly all NA
    RPMdf = pd.DataFrame.dropna(RPMdf,axis='columns',thresh=10)

    #add index of last row of database to RPM dataframe to bin correctly
    lastIndex = RPMdf.iloc[[-1]].index.values[0]

    #get index of RPMs to group them by their RPM
    RPMindex = np.append(RPMindex,lastIndex)
    RPMlabel = RPMs[a[0]].to_list()

    #label columns as what they're portraying
    #assumes all bins of RPM have same values at same columns
    #assumes V is in first column, find it
    Units = RPMdf[RPMdf[0].str.match('V')]

    #find first row V is in
    unitsRow = Units[0].index[0]

    #take first row and set this as column headers
    RPMdf.columns = RPMdf.loc[unitsRow]

    #remove all non-numeric rows
    RPMdf = RPMdf.apply(lambda x: pd.to_numeric(x, errors='coerce')).dropna()

    #finally group each RPM
    RPMdf = RPMdf.groupby(pd.cut(RPMdf.index, RPMindex, right=False, labels = RPMlabel))

    #for printing out grouped dataframe
    #for name,group in RPMdf:
    #   print(name)
    #   print(group)


    #NOTES#
    #J = Velocity / (RPs*Diameter (m))
    """
    Some Shortened terms

    Curr = Current
    Res = Resistance
    Batt = Battery
    Volt = Voltage
    v = Voltage (when put before a term)
    p = Power (when put before a term)
    Amps = Amperage
    """

    propDiameter = propDiameter*0.0254     #convert prop diameter to meters

    #WARNING --- pressure and altitude calcaulator only valid to approximately 12,000 m
    if(selectAltitude):
        h = altitude/3.281 #convert to meters
        Temperature = 15+273.15-6.5*h/1000 #temp in Kelvin
        T = Temperature 
        R = 8.3143       #constant
        M = 0.02896      #molar mass of Earth's air
        g = 9.806        #gravity
        Po = 101325      #sea level pressure in kPa
        Pressure = Po*np.exp(-M*g*h/(R*T))    #obtained Pressure in kPa
        Density = (Pressure*M)/(R*T)          #unit is in kg/m^3
    else:
        Density = density
    
    #-----------------------------------------------------------------------------------------------

    #Note ---- Prof. Merret at UIUC helped provide these formulas, comments added by me for the most part to try to explain each term

    RPMgroups = np.array(list(RPMdf.groups))
    allAmps = []
    for groups in RPMgroups:
        RPMgroup = RPMdf.get_group(groups)
        PWR = RPMgroup['PWR'].iloc[0]*745.7
        allAmps.append(PWR/(cells*battVolt))
    #allAmps is an array with expected amp draw at max static thrust at each velocity 
    #print(allAmps)
    velocityArray = [0]

    def RPMfinder(motorAmpGuess,battAmpGuess,Output,velocity):

        #main calulation for iteration of pMotorOut
        def pMotorOutCalc(motorAmp,battAmp):

            vBatt = cells*battVolt-cells*battRes*battAmp   #get total battery volts with voltage loss
            vTerm = (vBatt-eRes*battAmp)*Throttle/100      #get voltage drop from esc and throttle value
            vDropMotor = motorAmp*mRes                     #get voltage drop from motor
            pMotorIn = vTerm*battAmp                       #get power motor uses in operation
            vEMF = vTerm-vDropMotor                        #get voltage motor actually has that after esc,battery,throttle losses
            pMotorOut = vEMF*(motorAmp-nLCurr)             #get power motor generates (voltage of motor*(Amps of motor-no load motor Amps))
            pMotorIn = vTerm*(motorAmp-nLCurr)             #get power motor receives

            return pMotorIn,pMotorOut,vTerm,motorAmp,vBatt,battAmp,vEMF

        pMotorIn,pMotorOut,vTerm,motorAmp,vBatt,battAmp,vEMF = pMotorOutCalc(motorAmpGuess,battAmpGuess)

        RPM = vEMF*kV                                  #get revolutions per minute of motor
        RPS = RPM/60                                   #get revolutions per second of motor
        #print('RPS: '+str(RPS))

        #checks if RPM is positive    
        if(RPM < 0):                  
            raise Exception("RPM is negative, something is wrong: " + str(RPM))

        #find closest data to RPM values, both above and below it
        RPMgroups = np.array(list(RPMdf.groups))
        if(RPM > RPMgroups.max()):
            raise Exception('RPM maximum limit reached, no data for higher RPM')
        if(RPM < RPMgroups.min()):
            raise Exception('RPM minimum limit reached, no data for lower RPM')
        bottomRPM = RPMgroups[RPMgroups < RPM].max()
        topRPM = RPMgroups[RPMgroups > RPM].min()

        #then get bottom and top RPM groups
        bottomRPMdata = RPMdf.get_group(bottomRPM).reset_index()
        topRPMdata = RPMdf.get_group(topRPM).reset_index()

        #then get cp and ct of both bottom and top RPM groups for desired velocity
        velocityMPH = velocity*2.23694

        #print('Velocity: ' + str(velocityMPH))
        #print('Index Velocity: ' + str(abs(bottomRPMdata['V']-velocityMPH).idxmin()))
        #print(bottomRPMdata)
        #find relevant indexes
        bottomRPMClosestVelocity = abs(bottomRPMdata['V']-velocityMPH).idxmin()
        topRPMClosestVelocity = abs(topRPMdata['V']-velocityMPH).idxmin()

        #find relevant ct
        bottomct = bottomRPMdata['Ct'].iloc[bottomRPMClosestVelocity]
        topct = topRPMdata['Ct'].iloc[topRPMClosestVelocity]

        #find relevant cp
        bottomcp = bottomRPMdata['Cp'].iloc[bottomRPMClosestVelocity]
        topcp = topRPMdata['Cp'].iloc[topRPMClosestVelocity]

        #find relevant torques
        bottomTorque = bottomRPMdata['Torque'].iloc[bottomRPMClosestVelocity]
        topTorque = topRPMdata['Torque'].iloc[topRPMClosestVelocity]

        #interpolate to get cp and ct for motor RPM
        cp = bottomcp+(RPM-bottomRPM)*(topcp-bottomcp)/(topRPM-bottomRPM)
        ct = bottomct+(RPM-bottomRPM)*(topct-bottomct)/(topRPM-bottomRPM)
        torque = bottomTorque+(RPM-bottomRPM)*(topTorque-bottomTorque)/(topRPM-bottomRPM)


        #prop power and thrust calculation
        pProp = Density*RPS**3*propDiameter**5*cp   #get power prop requires in kg*m^2/s^3  (W)
        ThrustProp = Density*RPS**2*propDiameter**4*ct   #get thrust prop generates in kg*m/s^2  (N)
        #in hP and english units
        #pPropEng = pProp*0.74                  #get power prop requires in ft-lb/s
        #pProphp = pProp*0.00134102             #get power prop requires in hp
        #tPropEng = ThrustProp*0.2248090795     #get thrust prop generates in lb

        if Output == 1:
            print('Export final parameters here')

        return pMotorIn,pMotorOut,pProp,vTerm,motorAmp,vBatt,battAmp,ThrustProp,vEMF,RPM,torque

    #first Amp guess
    ImotorGuess = nLCurr+0.01 #make first motor amp draw guess as no load amp draw for motor
    IbattGuess = nLCurr+0.01

    #dataframe for storing final results
    resultProp = pd.DataFrame(columns=[ "Battery Current (A)","RPM (rev/min)", "Thrust (N)","Power (W)","Efficiency", "Torque (in-lbf)"])
    resultProp.index.name = 'Velocity (m/s)'


    powerTolerance = 1 #how close the power out of the motor should be to the power out of the propeller

    velocityPosition = 0 #velocityArray position
    velocity = velocityArray[velocityPosition] #current velocity in m/s

    #have to run here for the first time to make while loop work
    pMotorIn,pMotorOut,pProp,vTerm,motorAmp,vBatt,battAmp,ThrustProp,vEMF,RPM,Torque = RPMfinder(ImotorGuess,IbattGuess,0,velocity)

    while(ThrustProp>0):

        if(velocity != 0):
            velocityArray.append(velocity)

        timeRun = 0 #variable to determine how often loop has run

        while(abs(pProp-pMotorOut)>powerTolerance):
            #print('pProp ' + str(pProp))
            #print('pMotorOut ' + str(pMotorOut))
            pMotorIn,pMotorOut,pProp,vTerm,motorAmp,vBatt,battAmp,ThrustProp,vEMF,RPM,Torque = RPMfinder(ImotorGuess,IbattGuess,0,velocity)
            #checks if motor power out is positive
            if(pMotorOut < 0):
                raise Exception("Motor Power Out is negative, something is wrong")
            pESCOut = pMotorIn
            vESCin = cells*battVolt-cells*battRes*battAmp
            vESCout = cells*battVolt-(cells*battRes+mRes)*battAmp
            ImotorGuess = pProp/vEMF+nLCurr
            IbattGuess = pESCOut/vESCout
            timeRun = timeRun+1
            #print('Iteration: ' + str(timeRun) + '     Velocity: '+ str(velocityArray[velocityPosition])+' m\s')

        #run one more time and store results in dataframe
        pMotorIn,pMotorOut,pProp,vTerm,motorAmp,vBatt,battAmp,ThrustProp,vEMF,RPM,Torque = RPMfinder(ImotorGuess,IbattGuess,0,velocity)
        #can keep previous motorAmp and battAmp values as they will be close to new ones most likely


        #checks if battery can supply current
        if(battAmp > battVolt/battRes):
            raise Exception("Batteries can't supply this much current: " + str(battAmp))

        #checks if ESC can handle Current
        if(battAmp > eCurrMax):
            raise Exception("ESC will be overloaded, choose larger ESC: " + str(battAmp)) 

        #print('motorAmp: '+str(motorAmp))
        #print('battAmp: '+str(battAmp))

        efficiency = pMotorOut/pMotorIn

        resultProp.loc[velocityArray[velocityPosition]] = [motorAmp,RPM,ThrustProp,pProp,efficiency,Torque]

        velocityPosition = velocityPosition+1
        velocity = velocity + velocityStep
        #print('New Velocity')

    #add thrust in pounds
    thrustLB = resultProp.loc[:,'Thrust (N)']*0.224809
    resultProp['Thrust (lbf)'] = thrustLB
    
    #add velocity in ft/s
    velocityFPS = np.round(resultProp.index.values*3.28084)
    #print(velocityFPS)
    resultProp.insert(0, 'Velocity (ft/s)',velocityFPS)
    
    #print motor name and propeller type
    print(motorName)
    print(propType)
    
    #drop last row because FN is 0 so results may be invalid
    resultProp.drop(resultProp.tail(1).index,inplace=True)
    display(resultProp)
    
    #locate and print out some useful variables
    ampsStatic = (str)(resultProp.loc[0,'Battery Current (A)'])
    thrustStatic = (str)(resultProp.loc[0,'Thrust (lbf)'])
    if(resultProp.shape[0] <= 16):
        ampsCruise80 = "NA"      
        thrustCruise80 = "NA" 
    else:
        ampsCruise80 = (str)(resultProp.loc[:, 'Battery Current (A)'].values[16])
        thrustCruise80 = (str)(resultProp.loc[:, 'Thrust (lbf)'].values[16])
    if(resultProp.shape[0] <= 20):
        ampsCruise100 = "NA"      
        thrustCruise100 = "NA" 
    else:
        ampsCruise100 = (str)(resultProp.loc[:, 'Battery Current (A)'].values[20])
        thrustCruise100 = (str)(resultProp.loc[:, 'Thrust (lbf)'].values[20])
    
    displayExcel = pd.DataFrame([propType,cells,ampsStatic,thrustStatic,ampsCruise80,thrustCruise80,ampsCruise100,thrustCruise100])
    displayExcel = displayExcel.transpose()
    displayExcel.columns = ['propType','cells','ampsStatic','thrustStatic','ampsCruise80','thrustCruise80','ampsCruise100','thrustCruise100']
    display(displayExcel)

HK-A40-10S_710kV_Batt4S
15x4E


,Velocity (ft/s),Battery Current (A),RPM (rev/min),Thrust (N),Power (W),Efficiency,Torque (in-lbf),Thrust (lbf)
Velocity (m/s),,,,,,,,
0.000,0.0,50.408264,9154.791294,39.046606,630.345716,0.920804,5.824150,8.778028
1.524,5.0,50.408264,9154.791294,37.294844,616.374078,0.920804,5.705720,8.384217
3.048,10.0,49.211711,9188.593947,36.624799,616.521577,0.922802,5.691957,8.233584
4.572,15.0,49.211711,9188.593947,34.615913,602.394603,0.922802,5.562716,7.781969
6.096,20.0,47.585768,9234.051350,33.105042,598.369400,0.925502,5.499395,7.442311
7.620,25.0,47.585768,9234.051350,30.782111,580.447302,0.925502,5.336033,6.920095
9.144,30.0,45.945827,9279.924727,29.994978,579.912720,0.928215,5.307180,6.743141
10.668,35.0,45.945827,9279.924727,27.463693,558.084084,0.928215,5.098823,6.174085
12.192,40.0,43.198747,9356.683925,25.808077,548.966231,0.932735,4.998791,5.801888


,propType,cells,ampsStatic,thrustStatic,ampsCruise80,thrustCruise80,ampsCruise100,thrustCruise100
0,15x4E,4,50.40826420986969,8.778028353296051,22.93424977965589,1.759974022375443,12.610450002969941,0.09843402960893269
